In [1]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv, lstsq

from scipy.optimize import minimize_scalar

from poly_basis import *
from spherepts import *
from rbf import *

from laplacebeltrami import *

from math import factorial as fac

In [2]:
n = 1000
eps = None
k = 50
rbf_obj = rbf_dict['multiquadric']
# rbf_obj = rbf_dict['r^9']
# Choose solution and forcing fuction
solution_index = 20

nodes = gen_spiral_nodes(n)
normals = nodes

sol_deg = sphere_harm_degs[solution_index]
exact = lambda x: sphere_harm[solution_index](*x)*-sol_deg*(sol_deg+1)
forcing = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sol_deg)

Harmonic degree: 4


# Shankar-Wright Method

In [3]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=2, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.449692
0.00023759235030184497


In [12]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=1, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.001904940403098113


In [13]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=4, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.0005002980927061657


In [3]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=-1, poly_type='s')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.821766
0.00030839577294799705


# Tangent Plane Method

In [7]:
C = TPM(nodes, normals, rbf_obj, eps, k, poly_deg=2)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.00048721016990151497


In [4]:
C = TPM(nodes, normals, rbf_obj, eps, k, poly_deg=-1)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon = 0.823559
7.10414912415893e-05


# Symmetric OGr

In [3]:
C = SOGr(nodes, normals, rbf_obj, eps, k, poly_deg=None, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.439866
0.00010560047226705281


In [4]:
C = SOGr(nodes, normals, rbf_obj, eps, k, poly_deg=0, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.439866
0.00010517675599315773


In [5]:
C = SOGr(nodes, normals, rbf_obj, eps, k, poly_deg=1, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.439866
0.00010400431625191328


In [6]:
C = SOGr(nodes, normals, rbf_obj, eps, k, poly_deg=2, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.439866
9.147184192494719e-05


In [12]:
C = SOGr(nodes, normals, rbf_obj, eps, k, poly_deg=5, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.439866
3.679303853167017e-11
